In [13]:
import scipy.io
import os
from random import sample
import numpy as np
import json
from numpy import *
import torch
from collections import Counter

In [2]:
# Read again processed files
x_train = np.load("../benchmark/RAW_DATA/PTBXL_SUPERCLASS/x_train.npy", allow_pickle=True)
y_train = np.load("../benchmark/RAW_DATA/PTBXL_SUPERCLASS/y_train.npy", allow_pickle=True)
x_test = np.load("../benchmark/RAW_DATA/PTBXL_SUPERCLASS/x_test.npy", allow_pickle=True)
y_test = np.load("../benchmark/RAW_DATA/PTBXL_SUPERCLASS/y_test.npy", allow_pickle=True)


In [3]:
Counter(y_train), Counter(y_test)

(Counter({0: 8157, 2: 2276, 4: 2158, 3: 479, 1: 1524}),
 Counter({0: 912, 2: 256, 1: 184, 4: 242, 3: 56}))

In [29]:
norm_id_train = np.where(y_train==0)
ls_norm_id = list(norm_id_train[0])
# ls_index = list(range(len(ls_norm_id)))
norm_id_delete_train = sample(ls_norm_id,6094)
len(norm_id_delete_train), norm_id_delete_train

(6094,
 [8847,
  589,
  8411,
  10392,
  12541,
  11873,
  630,
  2695,
  7200,
  8510,
  7961,
  13885,
  8115,
  781,
  11008,
  9273,
  4303,
  1606,
  607,
  11367,
  3994,
  4073,
  2617,
  3999,
  4190,
  2413,
  6025,
  11003,
  2445,
  1473,
  9421,
  5140,
  10360,
  7001,
  7742,
  7374,
  11893,
  7484,
  13236,
  14279,
  7116,
  11398,
  13864,
  7930,
  7983,
  9065,
  25,
  11702,
  90,
  13505,
  14071,
  2372,
  3294,
  7297,
  1795,
  12166,
  2929,
  624,
  8285,
  11918,
  12259,
  12947,
  8052,
  381,
  13582,
  9418,
  3345,
  9126,
  2521,
  9267,
  4460,
  4304,
  2111,
  11585,
  9789,
  7197,
  3919,
  11632,
  8113,
  6249,
  2426,
  207,
  2963,
  8472,
  8629,
  757,
  10619,
  5292,
  12957,
  766,
  3212,
  3148,
  2883,
  6632,
  6406,
  3516,
  12964,
  7344,
  3237,
  8348,
  10177,
  6550,
  5516,
  6780,
  4366,
  86,
  3282,
  561,
  6985,
  6837,
  6085,
  1188,
  2782,
  14133,
  9477,
  145,
  4625,
  11220,
  7084,
  10202,
  7731,
  1178,
  12

In [30]:
keep_ids = [i for i in range(x_train.shape[0]) if i not in norm_id_delete_train]
len(keep_ids)

8500

In [34]:
new_x_train = x_train[keep_ids,:,:]

In [35]:
new_x_train.shape

(8500, 1000, 12)

In [47]:
def convert_to_4_classes (x_np, y_np, remove_ele):
    norm_id_train = np.where(y_np==0)
    ls_norm_id = list(norm_id_train[0])
    norm_id_delete = sample(ls_norm_id,remove_ele)
    
    hyp_id_train = np.where(y_np==3)
    ls_hyp_delete = list(hyp_id_train[0])
    
    keep_ids = [i for i in range(x_np.shape[0]) if i not in norm_id_delete and i not in ls_hyp_delete]
    new_x = x_np[keep_ids,:,:]
    new_y = y_np[keep_ids]
    new_y[new_y==4] = 3
    return new_x, new_y

In [49]:
new_x_train, new_y_train = convert_to_4_classes(x_train, y_train, 5615)
new_x_test, new_y_test = convert_to_4_classes(x_test, y_test, 594)
Counter(new_y_train), Counter(new_y_test)

(Counter({0: 2542, 2: 2276, 3: 2158, 1: 1524}),
 Counter({0: 318, 2: 256, 1: 184, 3: 242}))

In [51]:
new_x_train.shape, new_x_test.shape

((8500, 1000, 12), (1000, 1000, 12))

In [53]:
np.save('../benchmark/RAW_DATA/PTBXL_4CLASSES/x_train.npy', np.array(new_x_train, dtype=object), allow_pickle=True)
np.save('../benchmark/RAW_DATA/PTBXL_4CLASSES/x_test.npy', np.array(new_x_test, dtype=object), allow_pickle=True)
np.save('../benchmark/RAW_DATA/PTBXL_4CLASSES/y_train.npy', np.array(new_y_train, dtype=object), allow_pickle=True)
np.save('../benchmark/RAW_DATA/PTBXL_4CLASSES/y_test.npy', np.array(new_y_test, dtype=object), allow_pickle=True)

In [55]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

scaler = StandardScaler()
scaler.fit(new_x_train.reshape(-1, 1))
with open('../benchmark/RAW_DATA/PTBXL_4CLASSES/standard_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)